# Import Statements

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Mapping packages:
import folium

In [7]:
# import geopandas as gpd

To install geopandas and folium:

`conda install -c conda-forge geopandas`

`conda install -c conda-forge folium `

then:

`jupyter nbextension enable vega --py --sys-prefix`

to enable the necessary notebook extension to use Folium.

# Importing Data

In [8]:
# Don't edit raw dataframes after they have been imported:
train_raw = pd.read_csv('assets/train.csv')
test_raw = pd.read_csv('assets/test.csv')
weather_raw = pd.read_csv('assets/weather.csv')
spray_raw = pd.read_csv('assets/spray.csv')

# These can be the cleaned versions:
train = train_raw.copy().drop(['Address','Block','Street','AddressNumberAndStreet','AddressAccuracy'], axis=1)
test = test_raw.copy().drop(['Address','Block','Street','AddressNumberAndStreet', 'AddressAccuracy'], axis=1)
# If we're using latitude and longitude we can drop out the address info.
weather = weather_raw.copy()
spray = spray_raw.copy()

# Cleaning Data and EDA, train and test

## Simple Map showing the Testing Locations

In [9]:
m = folium.Map(
    location=[41.883844, -87.632162],
    tiles='openstreetmap',
    zoom_start=10,
    max_zoom = 10,
    min_zoom = 10
    
)



#folium.tooltip = 'Click Me!' #WARNING: Tooltips are not in this release; use the dev one if you want it



folium.Marker([41.973312, -87.910576], popup='OHare Iternational Airport', icon=folium.Icon(icon='plane', color = 'red')).add_to(m)
folium.Marker([41.786, -87.752], popup='Midway International Airport', icon=folium.Icon(icon='plane', color = 'green')).add_to(m)

m

## Examining the training set:

In [10]:
train.head()

,Date,Species,Trap,Latitude,Longitude,NumMosquitos,WnvPresent
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,1,0
1,2007-05-29,CULEX RESTUANS,T002,41.954690,-87.800991,1,0
2,2007-05-29,CULEX RESTUANS,T007,41.994991,-87.769279,1,0
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974089,-87.824812,1,0
4,2007-05-29,CULEX RESTUANS,T015,41.974089,-87.824812,4,0


### Exploring Species Column -- Ben

In [11]:
train['Species'].value_counts() # WNV is transmitted by all mosquitos in the Culex genus.

CULEX PIPIENS/RESTUANS    4752
CULEX RESTUANS            2740
CULEX PIPIENS             2699
CULEX TERRITANS            222
CULEX SALINARIUS            86
CULEX TARSALIS               6
CULEX ERRATICUS              1
Name: Species, dtype: int64

[Which mosquitos spread WNV?](https://wwwnc.cdc.gov/eid/article/7/6/01-0617_article)

In [12]:
# In the training dataset, only Culex Pipiens and Culex Restuans carried WNV.
train.groupby(by ='Species')['NumMosquitos','WnvPresent'].mean()

,NumMosquitos,WnvPresent
Species,,
CULEX ERRATICUS,7.000000,0.000000
CULEX PIPIENS,16.550945,0.088922
CULEX PIPIENS/RESTUANS,13.945286,0.055135
CULEX RESTUANS,8.551460,0.017883
CULEX SALINARIUS,1.686047,0.000000
CULEX TARSALIS,1.166667,0.000000
CULEX TERRITANS,2.297297,0.000000


# Cleaning Data and EDA, weather -- Steve

In [13]:
weather = pd.read_csv('assets/Proj4WeatherWithLoc.csv')
# Steve has created a new .csv with location data for the weather stations

In [14]:
weather.columns

Index(['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth',
       'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure (in hg)',
       'SeaLevel (in hg)', 'ResultSpeed', 'ResultDir', 'AvgSpeed'],
      dtype='object')

In [15]:
weather['Station'] = weather['Station'].astype(str)

#Converting to string so I can run the Map function below.

weather["Station Location"] = weather["Station"].map(lambda x: "O'HARE" if '1' in x else "MIDWAY" if '2' in x else "")

In [16]:
weather['Station'] = weather['Station'].astype(int)
#And back to integer now for math usage

# O'Hare: 41.970748, -87.908336
# Midway: 41.787957, -87.752359

weather["Latitude"] = weather["Station Location"].map(lambda x: "41.970748" if "O'HARE" in x else "41.787957" if "MIDWAY" in x else "")
weather["Longitude"] = weather["Station Location"].map(lambda x: "-87.908336" if "O'HARE" in x else "-87.752359" if "MIDWAY" in x else "")

In [17]:
#Mapping latitude and longitude for the airports

weather["Latitude"] = weather["Latitude"].astype(float)
weather["Longitude"] = weather["Longitude"].astype(float)

#Converting the string entries of location back into floats for math usage

In [18]:
weather['Date'] = pd.to_datetime(weather['Date'], format='%Y/%m/%d')

weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,SnowFall,PrecipTotal,StnPressure (in hg),SeaLevel (in hg),ResultSpeed,ResultDir,AvgSpeed,Station Location,Latitude,Longitude
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,0,0,29.1,29.82,1.7,27,9.2,O'HARE,41.970748,-87.908336
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,M,0,29.18,29.82,2.7,25,9.6,MIDWAY,41.787957,-87.752359
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,0,0,29.38,30.09,13.0,4,13.4,O'HARE,41.970748,-87.908336
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,M,0,29.44,30.08,13.3,2,13.4,MIDWAY,41.787957,-87.752359
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,0,0,29.39,30.12,11.7,7,11.9,O'HARE,41.970748,-87.908336


In [19]:
weather.Date.dtype

dtype('<M8[ns]')

In [20]:
weather2011 = weather.loc[(weather['Date'] > '2011-05-01') & (weather['Date'] <= '2011-10-31')]
weather2013 = weather.loc[(weather['Date'] > '2013-05-01') & (weather['Date'] <= '2013-10-31')]

#Making a subset of the weather data over just these years as thats all the spray data we have.

weather2011.reset_index(drop=True)
weather2013.reset_index(drop=True)

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,SnowFall,PrecipTotal,StnPressure (in hg),SeaLevel (in hg),ResultSpeed,ResultDir,AvgSpeed,Station Location,Latitude,Longitude
0,1,2013-05-02,55,42,49,-5,41,45,16,0,...,0,0.11,29.44,30.17,15.2,3,15.4,O'HARE,41.970748,-87.908336
1,2,2013-05-02,71,42,M,M,39,45,M,M,...,M,0,29.51,30.17,15.8,2,16.1,MIDWAY,41.787957,-87.752359
2,1,2013-05-03,48,40,44,-10,42,43,21,0,...,0,0.05,29.28,30.03,11.2,4,11.5,O'HARE,41.970748,-87.908336
3,2,2013-05-03,50,40,45,M,41,44,20,0,...,M,0.02,29.34,30.03,11.0,2,11.2,MIDWAY,41.787957,-87.752359
4,1,2013-05-04,71,48,60,6,45,51,5,0,...,0,0.02,29.26,29.99,6.1,6,7,O'HARE,41.970748,-87.908336
5,2,2013-05-04,72,49,61,M,44,51,4,0,...,M,T,29.32,29.99,6.0,6,7.6,MIDWAY,41.787957,-87.752359
6,1,2013-05-05,69,48,59,4,43,50,6,0,...,0,T,29.28,30,8.8,3,9.3,O'HARE,41.970748,-87.908336
7,2,2013-05-05,70,49,60,M,41,50,5,0,...,M,0,29.34,30,8.0,2,8.6,MIDWAY,41.787957,-87.752359
8,1,2013-05-06,71,50,61,6,39,49,4,0,...,0,T,29.28,30,7.5,3,8,O'HARE,41.970748,-87.908336
9,2,2013-05-06,70,51,61,M,37,49,4,0,...,M,0,29.34,30,7.0,2,7.5,MIDWAY,41.787957,-87.752359


# Cleaning Data and EDA, spray

## Steve's work mapping the spray data:

In [22]:
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [27]:
spray['CombinedLoc'] = spray.Latitude.astype(str).str.cat(spray.Longitude.astype(str), sep=', ')

spray.Date.dtype

spray['Date'] = pd.to_datetime(spray['Date'], format='%Y/%m/%d')

#2011: 08-29 to 09/07
#2013: 07-17 to 09/05

spray2011 = spray.loc[(spray['Date'] > '2011-08-29') & (spray['Date'] <= '2011-09-07')]
spray2013 = spray.loc[(spray['Date'] > '2013-07-17') & (spray['Date'] <= '2013-09-05')]

spray2011 = spray2011.reset_index(drop=True)
spray2013 = spray2013.reset_index(drop=True)

In [28]:
spray2011.head()

,Date,Time,Latitude,Longitude,CombinedLoc
0,2011-09-07,7:52:52 PM,41.981433,-87.787777,"41.9814333333, -87.7877766667"
1,2011-09-07,7:53:02 PM,41.980998,-87.787778,"41.9809983333, -87.7877783333"
2,2011-09-07,7:53:12 PM,41.980560,-87.787762,"41.98056, -87.7877616667"
3,2011-09-07,7:53:22 PM,41.980198,-87.787758,"41.9801983333, -87.7877583333"
4,2011-09-07,7:53:32 PM,41.979752,-87.787765,"41.9797516667, -87.787765"


### Spraying locations in 2011:

In [29]:
from folium import plugins
from folium.plugins import MarkerCluster

location2011 = spray2011['Latitude'].mean(), spray2011['Longitude'].mean()
# locationlist2011 = spray2011[["Latitude","Longitude"]].values.tolist()

map2011 = (folium.Map(location=location2011,zoom_start=13))

for index,row in spray2011.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']],
                        radius=3,
                        fill = True,
                        color = "#4286f4", 
                        fill_color="#4286f4"
                       ).add_to(map2011)

map2011.save('2011 Spray.html') 

### Spraying locations in 2013:

In [30]:
map2013 = (folium.Map(location=[41.977049, -87.768738],zoom_start=11))

for index,row2 in spray2013.iterrows():
    folium.CircleMarker([row2['Latitude'], row2['Longitude']],
                       radius=3,
                        fill = True,
                        color = "#b22323", 
                        fill_color="#b22323", # divvy color
                       ).add_to(map2013)

map2013.save('2013 Spray.html')

### All Spraying Locations (2011 & 2013), clustered together

In [31]:
masterspraycluster = (folium.Map(location=[41.843667, -87.803933],zoom_start=10))

marker_cluster = MarkerCluster().add_to(masterspraycluster)

for index,row in spray2011.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']], 
                        radius=3,
                        fill = True,
                        color = "#38ff55", 
                        fill_color="#38ff55").add_to(marker_cluster)
                       

for index, row2 in spray2013.iterrows():
    folium.CircleMarker([row2['Latitude'], row2['Longitude']], 
                        radius=3,
                        fill=True, 
                        color="#ff0000", 
                        fill_color= "#ff0000").add_to(marker_cluster)

masterspraycluster.add_child(folium.LatLngPopup())
masterspraycluster.save('Master Spray (with Clusters).html')

### All Spraying Locations (2011 & 2013), Raw

In [32]:
masterspray = (folium.Map(location=[41.843667, -87.803933],zoom_start=10))

for index,row in spray2011.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']], 
                        radius=3,
                        fill = True,
                        color = "#38ff55", 
                        fill_color="#38ff55").add_to(masterspray)
                       

for index, row2 in spray2013.iterrows():
    folium.CircleMarker([row2['Latitude'], row2['Longitude']], 
                        radius=3,
                        fill=True, 
                        color="#ff0000", 
                        fill_color= "#ff0000").add_to(masterspray)

masterspray.add_child(folium.LatLngPopup())
masterspray.save('Master Spray.html')

In [33]:
# markerspray = (folium.Map(location=[41.843667, -87.803933],zoom_start=10))

# locationlist2011 = spray2011[["Latitude","Longitude"]].values.tolist()
# locationlist2013 = spray2013[["Latitude","Longitude"]].values.tolist()
# icon2011 = folium.Icon(color='red',icon='ok-sign')
# icon2013 = folium.Icon(color='green',icon='ok-sign')

# for index,row in spray2011.iterrows():
#     folium.Marker([row['Latitude'], row['Longitude']], icon=icon2011,
#                   popup=str(row['Date'], 
#                   )).add_to(markerspray)
                       

# for index, row2 in spray2013.iterrows():
#     folium.Marker([row2['Latitude'], row2['Longitude']], icon=icon2013, 
#                   popup=str(row2['Date'] 
#                   )).add_to(markerspray)
    
# markerspray.save('Marker Spray.html')
   

    
#This Currently does not work.  I'll try and figure it out.